
# Space

In [1]:
import os
import sys 
import logging
import pandas as pd 
from pprint import pprint 

# WorkSpace
KEY = 'WorkSpace'; WORKSPACE_PATH = os.getcwd().split(KEY)[0] + KEY; print(WORKSPACE_PATH)
os.chdir(WORKSPACE_PATH)
sys.path.append(WORKSPACE_PATH)

# Pipeline Space
from proj_space import SPACE
SPACE['WORKSPACE_PATH'] = WORKSPACE_PATH
sys.path.append(SPACE['CODE_FN'])
pprint(SPACE)

# Available Packages
import pandas as pd
from datetime import datetime 

logger = logging.getLogger(__name__)
recfldtkn_config_path = os.path.join(SPACE['CODE_FN'], 'config_recfldtkn/')


g:\Shared drives\CDHAI-WellDoc\2024-WellDocTest-SPACE\_WellDoc-AI-CGMGPT-WorkSpace
{'CODE_FN': '../pipeline',
 'CODE_RFT': '../pipeline',
 'DATA_CaseFeat': '../_Data/3-Data_CaseFeat',
 'DATA_CaseObs': '../_Data/2-Data_CaseObs',
 'DATA_CaseSet': '../_Data/4-Data_CaseSet',
 'DATA_RAW': '../_Data/0-Data_Raw',
 'DATA_RFT': '../_Data/1-Data_RFT',
 'DATA_TASK': 'Data_CGMGPT',
 'MODEL_TASK': 'Model_CGMGPT',
 'WORKSPACE_PATH': 'g:\\Shared '
                   'drives\\CDHAI-WellDoc\\2024-WellDocTest-SPACE\\_WellDoc-AI-CGMGPT-WorkSpace'}


# Dataset

In [2]:
import datasets
from recfldtkn.loadtools import load_ds_rec_and_info
from recfldtkn.configfn import load_cohort_args, load_record_args

base_config = load_cohort_args(recfldtkn_config_path, SPACE)
print(base_config)

{'CohortInfo': {'RawData2022_CGM': {'cohort_label': 1, 'cohort_name': 'RawData2022_CGM', 'FolderPath': '../_Data/0-Data_Raw/RawData2022_CGM/'}, 'RawData2023_CVSTDCAug': {'cohort_label': 2, 'cohort_name': 'RawData2023_CVSTDCAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSTDCAug/'}, 'RawData2023_CVSDeRxAug': {'cohort_label': 3, 'cohort_name': 'RawData2023_CVSDeRxAug', 'FolderPath': '../_Data/0-Data_Raw/RawData2023_CVSDeRxAug/'}}, 'RawRootID': 'PatientID', 'RootID': 'PID', 'RecName': 'PRawRecNum', 'recattr_pyfolder': '../pipeline/fn_recattr/', 'fldtkn_pyfolder': '../pipeline/fn_fldtkn/', 'humanrec_pyfolder': '../pipeline/fn_humanrec/', 'inference_pyfolder': '../pipeline/fn_inference/', 'trigger_pyfolder': '../pipeline/fn_trigger/', 'RecName_to_RFT_GROUP_SIZE': {'CGM5Min': 100, 'Default': 5000}, 'RecName_to_RFT_idx_group_size': {'CGM5Min': 100, 'BGManual': 100, 'Default': 10000}, 'RecName_to_RFT_usebucket': {'CGM5Min': True, 'Default': False}, 'RootIDLength': 6, 'recfldtkn_config_p

# TriggerCaseMethod

In [3]:
from recfldtkn.pipeline_model import get_Trigger_Cases, convert_TriggerCases_to_LearningCases

use_learning = True 
use_inference = not use_learning

######################################
TriggerCaseMethod = 'CGM5MinEntry'
cohort_label_list = [1]
Trigger2LearningMethods = [
    
    {'op':'Tag',    'Name': 'Tag_PttBasicWD'},
    {'op':'Filter', 'Name': 'Filter_PttBasicWD'},
    
    {'op':'CFQ',    'Name': 'CFQ_Bf24hCGMrn'},
    {'op':'TagCF',  'Name': 'TagCF_Bf24hCGMrn', 'CFQName': 'CFQ_Bf24hCGMrn'},
    
    {'op':'CFQ',    'Name': 'CFQ_Af2hCGMrn',    
     'type': 'learning-only'},
    {'op':'TagCF',  'Name': 'TagCF_Af2hCGMrn',  'CFQName': 'CFQ_Af2hCGMrn',  
     'type': 'learning-only',},
    
    # {'op':'Filter', 'Name': 'Filter_BfCGMgeq280'},
    # {'op':'Filter', 'Name': 'Filter_AfCGMgeq24',  
    #  'type': 'learning-only'},
    
    # {'op':'CFQ',    'Name': 'CFQ_Bf1mMEDALrn',    
    #  'type': 'learning-only'},
    # {'op':'TagCF',  'Name': 'TagCF_Bf1mMEDALrn', 'CFQName': 'CFQ_Bf1mMEDALrn', 
    #  'type': 'learning-only'},

]
######################################

In [4]:
use_inference

False

In [5]:
from recfldtkn.loadtools import fetch_casetag_tools, fetch_casefilter_tools, load_module_variables

def convert_TriggerCases_to_LearningCases(df_case, 
                                          cohort_label_list,
                                          Trigger2LearningMethods, 
                                          base_config, 
                                          use_inference):
    
    CFQ_to_CaseFeatInfo = {}

    if use_inference == True:
        Trigger2LearningMethods = [i for i in Trigger2LearningMethods if i.get('type', None) != 'learning-only']
       
    # print(Trigger2LearningMethods)
    SPACE = base_config['SPACE']
    for method in Trigger2LearningMethods:
        if method['op'] == 'Tag':
            name = method['Name']
            logger.info(f'CaseTag: {name}')
            CaseTag_Tools = fetch_casetag_tools(name, SPACE)

            subgroup_columns = CaseTag_Tools['subgroup_columns']
            if 'InfoRecName' in CaseTag_Tools:
                InfoRecName = CaseTag_Tools['InfoRecName']
                ds_info, _ = load_ds_rec_and_info(InfoRecName, base_config, cohort_label_list)
            else:
                ds_info = None

            fn_case_tagging = CaseTag_Tools['fn_case_tagging']
            df_case = fn_case_tagging(df_case, ds_info, subgroup_columns, base_config)

        elif method['op'] == 'Filter':
            name = method['Name']
            logger.info(f'CaseFilter: {name}')
            CaseFilter_Tools = fetch_casefilter_tools(name, SPACE)
            fn_case_filtering = CaseFilter_Tools['fn_case_filtering']
            
            logger.info(f'Before Filter: {df_case.shape}')
            df_case = fn_case_filtering(df_case)
            logger.info(f'After Filter: {df_case.shape}')

        elif method['op'] == 'CFQ':
            name = method['Name']
            pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{name}.py')
            module = load_module_variables(pypath)
            fn_casefeat_querying = module.fn_casefeat_querying
            df_case, CaseFeatInfo = fn_casefeat_querying(df_case, base_config)
            # CaseFeatName = CaseFeatInfo['CaseFeatName']
            CFQ_to_CaseFeatInfo[name] = CaseFeatInfo

        elif method['op'] == 'TagCF':
            name = method['Name']
            pypath = os.path.join(SPACE['CODE_FN'], 'fn_learning', f'{name}.py')
            module = load_module_variables(pypath)
            fn_case_tagging_on_casefeat = module.fn_case_tagging_on_casefeat
            
            CFQName = method['CFQName']
            CFQ_to_CaseFeatInfo = CFQ_to_CaseFeatInfo[CFQName]
            df_case = fn_case_tagging_on_casefeat(df_case, CaseFeatInfo)

        else:
            raise ValueError(f'Unknown method: {method}')

    return df_case

In [6]:
RecName_to_dsRec = {}
RecName_to_dsRecInfo = {}

df_case = get_Trigger_Cases(TriggerCaseMethod, 
                            cohort_label_list, 
                            base_config, 
                            SPACE, 
                            RecName_to_dsRec, 
                            RecName_to_dsRecInfo)

#####################
# df_case = df_case.sample(1000, random_state=42)
#####################

df_case.shape

(17501383, 2)

# Split DataFrame

In [7]:
# Function to split DataFrame into chunks
def split_dataframe(df, chunk_size):
    chunks = []
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

In [8]:
len(df_case)

df_case = df_case[:1000]

In [9]:
# SIZE = 1_000_000
SIZE = 5_10
chunks = split_dataframe(df_case, SIZE)
print(len(chunks))

2


In [16]:
def process_chunk(chunk):
    try:
        df_case = convert_TriggerCases_to_LearningCases(chunk, 
                                                    cohort_label_list, 
                                                    Trigger2LearningMethods, 
                                                    base_config, 
                                                    use_inference)
        columns = [i for i in df_case.columns if '_co.' not in i]
        # print(columns)
        df_case = df_case[columns].reset_index(drop=True)
        # df_case.head()
        return df_case
    except Exception as e:
        print(f"Error processing chunk: {e}")
        raise  # Re-raise the exception to ensure it's visible outside the function

In [17]:
chunk = chunks[0]
chunk

,PID,ObsDT
0,1000006,2021-01-07 06:20:00
1,1000006,2021-01-07 06:25:00
2,1000006,2021-01-07 06:30:00
3,1000006,2021-01-07 06:35:00
4,1000006,2021-01-07 06:40:00
...,...,...
505,1000006,2021-01-23 14:10:00
506,1000006,2021-01-23 14:15:00
507,1000006,2021-01-23 14:20:00
508,1000006,2021-01-23 14:25:00


In [18]:
process_chunk(chunk)

[INFO:2024-03-04 01:02:30,619:(2962776366.py@19 __main__)]: CaseTag: Tag_PttBasicWD
[INFO:2024-03-04 01:02:30,660:(2962776366.py@34 __main__)]: CaseFilter: Filter_PttBasicWD
[INFO:2024-03-04 01:02:30,660:(2962776366.py@38 __main__)]: Before Filter: (510, 7)
[INFO:2024-03-04 01:02:30,661:(2962776366.py@40 __main__)]: After Filter: (510, 7)
[INFO:2024-03-04 01:02:38,797:(observer.py@892 recfldtkn.observer)]: provided casefeat_ids num: 510
[INFO:2024-03-04 01:02:38,804:(observer.py@582 recfldtkn.observer)]: provided caseobs_ids num: 510


Map:   0%|          | 0/510 [00:00<?, ? examples/s]

[INFO:2024-03-04 01:02:42,674:(observer.py@892 recfldtkn.observer)]: provided casefeat_ids num: 510
[INFO:2024-03-04 01:02:42,682:(observer.py@582 recfldtkn.observer)]: provided caseobs_ids num: 510


Map:   0%|          | 0/510 [00:00<?, ? examples/s]

,PID,ObsDT,Gender,YearOfBirth,DiseaseType,cohort,Age,BfCGMrn,AfCGMrn
0,1000006,2021-01-07 06:20:00,1,1978,1,C1,43,1.0,24.0
1,1000006,2021-01-07 06:25:00,1,1978,1,C1,43,2.0,24.0
2,1000006,2021-01-07 06:30:00,1,1978,1,C1,43,3.0,24.0
3,1000006,2021-01-07 06:35:00,1,1978,1,C1,43,4.0,24.0
4,1000006,2021-01-07 06:40:00,1,1978,1,C1,43,5.0,24.0
...,...,...,...,...,...,...,...,...,...
505,1000006,2021-01-23 14:10:00,1,1978,1,C1,43,32.0,24.0
506,1000006,2021-01-23 14:15:00,1,1978,1,C1,43,33.0,24.0
507,1000006,2021-01-23 14:20:00,1,1978,1,C1,43,34.0,24.0
508,1000006,2021-01-23 14:25:00,1,1978,1,C1,43,35.0,24.0


In [19]:
import os

# Determine the number of available CPUs
num_cpus = os.cpu_count()
num_cpus

24

In [24]:
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor, as_completed


num_workers = 2
with ProcessPoolExecutor() as executor:
    futures = [executor.submit(process_chunk, chunk) for chunk in chunks]
    for future in as_completed(futures):
        try:
            ds_case = future.result()
            print(ds_case)  # Or any other logic to handle the result
        except Exception as e:
            print(f"Error processing future: {e}")

Error processing future: A process in the process pool was terminated abruptly while the future was running or pending.
Error processing future: A process in the process pool was terminated abruptly while the future was running or pending.
